In [2]:
from langgraph_sdk import get_client

# Replace this with the URL of your own deployed graph
URL = "http://localhost:8123"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [28]:
import json

thread = await client.threads.create()
input_dict = {"request": 
              "i want to create an order in the system. customer id is 996778, delivery RETIRA EN FÁBRICA, and pays with EFECTIVO. you don't need those other optional fields, send them as empty strings."}

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="task_manager",
    input=input_dict,
    stream_mode="updates",):
    #    continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efc3a3f-e444-6653-9d22-48443ae12f43",
    "attempt": 1
}



Receiving new event of type: updates...
{
    "start": {
        "stored_procedures": {
            "procedures": [
                {
                    "description": "random test procedure [DO NOT USE]",
                    "actions": [
                        {
                            "id": 9,
                            "type": "erp",
                            "name": "select the system url",
                            "confirmed": true,
                            "params_schema": {
                                "title": "URLParams",
                                "description": "Parameters for sending an order to a URL",
                                "type": "object",
                                "properties": {
                                    "url": {
                                        "type": "string",
                                  

In [29]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="task_manager",
    input=None,
    stream_mode="values",):
        #continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efc3a40-2100-6f75-9e88-4b6cc4c79648",
    "attempt": 1
}



Receiving new event of type: values...
{
    "messages": [],
    "request": "i want to create an order in the system. customer id is 996778, delivery RETIRA EN F\u00c1BRICA, and pays with EFECTIVO. you don't need those other optional fields, send them as empty strings.",
    "stored_procedures": {
        "procedures": [
            {
                "description": "random test procedure [DO NOT USE]",
                "actions": [
                    {
                        "id": 9,
                        "type": "erp",
                        "name": "select the system url",
                        "confirmed": true,
                        "params_schema": {
                            "title": "URLParams",
                            "description": "Parameters for sending an order to a URL",
                            "type": "object",
                          

In [30]:
current_state = await client.threads.get_state(thread['thread_id'])
current_state['next']

['human_feedback_action']

In [3]:
current_state = await client.threads.get_state("e9124db3-e217-4cc7-ad00-ef41de57711a")
current_state['next']

['human_feedback_action']

In [31]:
current_state['tasks'][0]#['interrupts']

{'id': 'c1f1ccb2-592f-506d-63f2-52611d238ad6',
 'name': 'human_feedback_action',
 'path': ['__pregel_pull', 'human_feedback_action'],
 'error': None,
 'interrupts': [{'value': 'human_feedback_select_message ',
   'resumable': False,
   'ns': None,
   'when': 'during'}],
 'checkpoint': None,
 'state': None,
 'result': None}

In [20]:
await client.threads.update_state(
    thread['thread_id'], 
    {"human_feedback_action_message": "yes"}, 
    #as_node="start"
    )

{'checkpoint': {'thread_id': 'c10c78ff-584e-4a28-aea5-9080b90dd216',
  'checkpoint_ns': '',
  'checkpoint_id': '1efc224b-ff7b-6cba-8008-e74102c16488'},
 'configurable': {'thread_id': 'c10c78ff-584e-4a28-aea5-9080b90dd216',
  'checkpoint_ns': '',
  'checkpoint_id': '1efc224b-ff7b-6cba-8008-e74102c16488'},
 'checkpoint_id': '1efc224b-ff7b-6cba-8008-e74102c16488'}

In [21]:
async for chunk in client.runs.stream(
    #thread_id=current_state['checkpoint']['thread_id'],
    #checkpoint_id=current_state['checkpoint']['checkpoint_id'],
    thread_id=thread['thread_id'],
    assistant_id="task_manager",
    input=None,#{"human_confirmation_message": "yes"},
    stream_mode="updates",):
    #    continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efc224c-2361-689b-b990-f7222b498de6",
    "attempt": 1
}



Receiving new event of type: updates...
{
    "human_feedback_action": null
}



Receiving new event of type: updates...
{
    "executor": {
        "messages": [
            {
                "content": "i want to create an order in the system. customer id is 996778, delivery RETIRA EN F\u00c1BRICA, and pays with EFECTIVO. you don't need those other optional fields, send them as empty strings.",
                "additional_kwargs": {},
                "response_metadata": {},
                "type": "human",
                "name": null,
                "id": "aec0fa7e-f0ef-40eb-8ff1-8d7a50b38139",
                "example": false
            },
            {
                "content": "I have selected the 'create new order in system' procedure because it allows you to create an order with the required fields you provided.",
                "additional_kwargs": {},
  

In [17]:
current_state = await client.threads.get_state(thread['thread_id'])
current_state['next']

['human_feedback_action']

In [26]:
current_state['tasks'][0]['interrupts']

[]

In [4]:

current_state

{'values': {'messages': [{'content': 'Hola, quiero cargar un pedido del sistema, por favor. El cliente es jorge botta. Entrega es retira en fábrica y paga con efectivo. Después, para los productos, quiero dos unidades del producto EG-GEN-M-10000-CG-TTA\n',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '9ff1b0a8-3ba9-467a-bbef-40648e7d2b04',
    'example': False},
   {'content': 'Elegí la primera opción para crear un nuevo pedido en el sistema, ya que es lo que necesitas para cargar el pedido de Jorge Botta.',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'ai',
    'name': None,
    'id': 'ee31b366-b08a-4908-b265-82f906220e86',
    'example': False,
    'tool_calls': [],
    'invalid_tool_calls': [],
    'usage_metadata': None},
   {'content': 'I confirm the procedure',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '0df66f09-31bd-4a92-950a-3b4

In [33]:
current_state['values']['messages']

[{'content': "i want to create an order in the system. customer id is 996778, delivery RETIRA EN FÁBRICA, and pays with EFECTIVO. you don't need those other optional fields, send them as empty strings.",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': 'bd275e36-970d-4288-b6fd-7daa25f95113',
  'example': False},
 {'content': 'He seleccionado el procedimiento para crear un nuevo pedido en el sistema, ya que se ajusta a tu solicitud de crear un pedido con la información del cliente y el método de pago.',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': '21dbbfff-2f34-4366-87b1-3ed7d2515418',
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': [],
  'usage_metadata': None},
 {'content': 'I confirm the procedure',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': 'a9c3cdbb-6db2-4b48-bb87-72675a5ecc5d',
  'example': False},
 {'content': 'The